In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data.csv')

In [3]:
RANDOM_STATE = 42

TRAIN_SIZE = 0.6
VAL_SIZE = 0.15
TEST_SIZE = 0.25

# Разбиение выборки

**Разбивать будем случайно во времени** (без отложенного теста), т.к. большая часть выборки сконцентрирована в ноябре 2023 

In [4]:
# проверка уникальности `item_id`
df['item_id'].nunique() == df.shape[0]

True

In [5]:
# отделяем test
train_val_ids, test_ids = train_test_split(
    df['item_id'],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=df['attr_value_name']
)

# отделяем train
train_ids, val_ids = train_test_split(
    df.loc[df['item_id'].isin(train_val_ids), 'item_id'],
    test_size=VAL_SIZE * (TRAIN_SIZE + VAL_SIZE + TEST_SIZE) / (TRAIN_SIZE + VAL_SIZE),
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=df.loc[df['item_id'].isin(train_val_ids),'attr_value_name']
)

In [6]:
df.loc[df['item_id'].isin(train_ids), 'sample_part'] = 'train'
df.loc[df['item_id'].isin(val_ids), 'sample_part'] = 'val'
df.loc[df['item_id'].isin(test_ids), 'sample_part'] = 'test'

print(df['sample_part'].value_counts(dropna=False))
print(df['sample_part'].value_counts(dropna=False, normalize=True))

sample_part
train    119606
test      49836
val       29902
Name: count, dtype: int64
sample_part
train    0.599998
test     0.250000
val      0.150002
Name: proportion, dtype: float64


In [7]:
# проверяем как сработала стратификация
for sample_part in df['sample_part'].value_counts(dropna=False).keys():
    mask_sp = df['sample_part'] == sample_part
    print(
        f"""{sample_part}:\n\t{df.loc[mask_sp, 'attr_value_name'].value_counts(dropna=False, normalize=True)}""",
        end='\n\n'
    )

train:
	attr_value_name
kosmeticheskii     0.464592
evro               0.248282
trebuet_remonta    0.190199
dizainerskii       0.096927
Name: proportion, dtype: float64

test:
	attr_value_name
kosmeticheskii     0.464584
evro               0.248274
trebuet_remonta    0.190204
dizainerskii       0.096938
Name: proportion, dtype: float64

val:
	attr_value_name
kosmeticheskii     0.464618
evro               0.248278
trebuet_remonta    0.190188
dizainerskii       0.096917
Name: proportion, dtype: float64



In [8]:
df[['item_id', 'sample_part']].to_csv('splitting_sample.csv', index=False)